# Building Agents That Use Code

This notebook is part of the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course), a free Course from beginner to expert, where you learn to build Agents.

![Agents course share](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png)

Alfred is planning a party at the Wayne family mansion and needs your help to ensure everything goes smoothly. To assist him, we'll apply what we've learned about how a multi-step `CodeAgent` operates.


## Let's install the dependencies and login to our HF account to access the Inference API

If you haven't installed `smolagents` yet, you can do so by running the following command:

In [1]:
!pip install smolagents -U

Defaulting to user installation because normal site-packages is not writeable


Let's also login to the Hugging Face Hub to have access to the Inference API.

In [2]:
import os
from huggingface_hub import login

login(new_session=False, token=os.getenv("HF_TOKEN"))

/home/vscode/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
user = !huggingface-cli whoami
username = user[0]
print(f"Logged in as: {username}")

Logged in as: gperdrizet


## Selecting a Playlist for the Party Using `smolagents`

An important part of a successful party is the music. Alfred needs some help selecting the playlist. Luckily, `smolagents` has got us covered! We can build an agent capable of searching the web using DuckDuckGo. To give the agent access to this tool, we include it in the tool list when creating the agent.

For the model, we'll rely on `InferenceClientModel`, which provides access to Hugging Face's [Inference API](https://huggingface.co/docs/api-inference/index). The default model is `"Qwen/Qwen2.5-Coder-32B-Instruct"`, which is performant and available for fast inference, but you can select any compatible model from the Hub.

Running an agent is quite straightforward:

In [4]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, InferenceClientModel

agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=InferenceClientModel())

agent.run("Search for the best music recommendations for a party at the Wayne's mansion.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Search for the best music recommendations for a party at the Wayne's mansion.                                   │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  party_music_results = web_search(query="best party music")                                                       
  wayne_mansion_music_results = web_search(query="music for Wayne's mansion party")                                
  print("Party Music Results:", party_music_results)                                                               
  print("Wayne's Mansion Music Results:", wayne_mansion_music_results)                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Party Music Results: ## Search Results

[The 75 Best Party Songs That Will Get Everyone Dancing - 
Gear4music](https://www.gear4music.com/blog/best-party-songs/)
The best party songs 1. "September" - Earth, Wind & Fire (1978) Quite possibly the best party song. An infectious 
mix of funk and soul, "September" is celebrated for its upbeat melody and "ba-dee-ya" chorus, making it a timeless 
dance favorite.

[The Top 100 Best Party Songs of All Time - 
LiveAbout](https://www.liveabout.com/top-best-party-songs-of-all-time-3248355)
A&M. Black Eyed Peas originally released this song as "Let's Get Retarded." Upon the re-editing of the lyrics, it 
became a smash hit, providing the soundtrack for a wide range of parties and celebrations including the 2004 
Democratic Party national convention.

[40 Best Party Songs | Songs To Dance To, Ranked By Our Editors - Time 
Out](https://www.timeout.com/music/best-party-songs)
Best party songs of all time. 40. 'Raingurl' by Yaeji. Photograph: Courtesy of the artist. Release date: 2017. A 
modern classic house track from the effortlessly cool DJ-producer Yaeji ...

[Top 200 Most Requested Songs - DJ Event Planner](https://djeventplanner.com/mostrequested.htm)
Top 30 Most Requested Songs For Wedding Party Dance. This list is compiled based on over 2 million song requests 
made using the DJ Event Planner song request system. Rank Artist Song Title; 1: T.I. Bring 'em Out: 2: Chris Brown:
Forever: 3: Earth, Wind and Fire: September: 4: Walk The Moon: Shut Up And Dance: 5:

[Best Songs To Party: DJ's Ultimate Party Songs Playlist - 
Top40Weekly.com](https://top40weekly.com/best-songs-to-party/)
Best Songs To Party: DJ's Ultimate Party Songs Playlist. If you're looking for the ultimate playlist to get your 
party started, look no further than this best party songs playlist. Whether you're hosting a birthday bash, a 
backyard barbecue, or just a night in with friends, the right music can make all the difference. From classic hits 
to ...

[The 101 Best Party Songs Of All Time - Music Grotto](https://www.musicgrotto.com/best-party-songs/)
A pounding beat and dramatic vocals make this song one of the best party songs to come out of Europe, and the 
song's music video features a strange and compelling premise of a dead man being reanimated and falling in love 
with a woman. Tjis Verwest, (a.k.a. Tiësto) composed the song with help from James Bell, Julia Karlsson, and Anton 
Rundberg.

[100 Best Party Songs: The Ultimate Party Playlist](https://audiocaptain.com/best-party-songs/)
Best 2000s Party Songs. Stylistically speaking, music that dominated the charts in the 2000s was nondescript.. 
There was not much new regarding musical directions. Club and hip hop dominated the charts while indie acts and 
some rock groups still had influence.

[Best Party Songs That Always Get People Dancing 2025](https://yourmusicdepot.com/party-songs/)
The Best Party Songs. When it comes to throwing an unforgettable party, the right music isn't just important - it's
essential. The perfect party song has that magical ability to transform a quiet room into an electric atmosphere, 
getting everyone from wallflowers to dance enthusiasts up on their feet.

[The 50 Best Music Playlists for Parties [2025] - Playlist 
Names](https://www.playlistnames.org/best-music-playlists-for-parties/)
Creating the perfect party playlist is an art form! Here's what you need to consider: 1. Flow and Energy. Start 
Slow: Begin with some chill tracks to set the mood. Build Up: Gradually increase the tempo and energy as the night 
progresses. Peak Time: Hit them with the bangers during the peak of the party to keep everyone dancing! 2. Audience
Engagement

[The Top 100 Best Party Songs of All Time - YouTube 
Music](https://music.youtube.com/playlist?list=PL10BZg10BCDQxKXg0qN_-Y6RwvfdDvJdy)
With the YouTube Music app, enjoy over 100 million songs at your fingertips, plus albums, playlists, remixes, music
videos, live performances, cover

[Step 1: Duration 4.04 seconds| Input tokens: 2,062 | Output tokens: 119]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Compile a list of recommended songs based on the search results                                                
  recommended_songs = [                                                                                            
      "September - Earth, Wind & Fire",                                                                            
      "Let's Get It Started - Black Eyed Peas",                                                                    
      "Shut Up and Dance - Walk The Moon",                                                                         
      "Raingurl - Yaeji",                                                                                          
      "Uptown Funk - Mark Ronson ft. Bruno Mars",                                                                  
      "Bohemian Rhapsody - Queen",                                                                                 
      "Sweet Child O' Mine - Guns N' Roses",                                                                       
      "Stairway to Heaven - Led Zeppelin",                                                                         
      "Hotel California - Eagles",                                                                                 
      "Back in Black - AC/DC",                                                                                     
      "Imagine - John Lennon",                                                                                     
      "Billie Jean - Michael Jackson",                                                                             
      "Like a Prayer - Madonna",                                                                                   
      "Smells Like Teen Spirit - Nirvana"                                                                          
  ]                                                                                                                
                                                                                                                   
  # Print the final list of recommended songs                                                                      
  print("Recommended Songs for Wayne's Mansion Party:", recommended_songs)                                         
                                                                                                                   
  final_answer(recommended_songs)                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Recommended Songs for Wayne's Mansion Party: ['September - Earth, Wind & Fire', "Let's Get It Started - Black Eyed 
Peas", 'Shut Up and Dance - Walk The Moon', 'Raingurl - Yaeji', 'Uptown Funk - Mark Ronson ft. Bruno Mars', 
'Bohemian Rhapsody - Queen', "Sweet Child O' Mine - Guns N' Roses", 'Stairway to Heaven - Led Zeppelin', 'Hotel 
California - Eagles', 'Back in Black - AC/DC', 'Imagine - John Lennon', 'Billie Jean - Michael Jackson', 'Like a 
Prayer - Madonna', 'Smells Like Teen Spirit - Nirvana']

Out - Final answer: ['September - Earth, Wind & Fire', "Let's Get It Started - Black Eyed Peas", 'Shut Up and Dance
- Walk The Moon', 'Raingurl - Yaeji', 'Uptown Funk - Mark Ronson ft. Bruno Mars', 'Bohemian Rhapsody - Queen', 
"Sweet Child O' Mine - Guns N' Roses", 'Stairway to Heaven - Led Zeppelin', 'Hotel California - Eagles', 'Back in 
Black - AC/DC', 'Imagine - John Lennon', 'Billie Jean - Michael Jackson', 'Like a Prayer - Madonna', 'Smells Like 
Teen Spirit - Nirvana']

[Step 3: Duration 7.41 seconds| Input tokens: 12,161 | Output tokens: 1,173]

['September - Earth, Wind & Fire',
 "Let's Get It Started - Black Eyed Peas",
 'Shut Up and Dance - Walk The Moon',
 'Raingurl - Yaeji',
 'Uptown Funk - Mark Ronson ft. Bruno Mars',
 'Bohemian Rhapsody - Queen',
 "Sweet Child O' Mine - Guns N' Roses",
 'Stairway to Heaven - Led Zeppelin',
 'Hotel California - Eagles',
 'Back in Black - AC/DC',
 'Imagine - John Lennon',
 'Billie Jean - Michael Jackson',
 'Like a Prayer - Madonna',
 'Smells Like Teen Spirit - Nirvana']

When you run this example, the output will **display a trace of the workflow steps being executed**. It will also print the corresponding Python code with the message:

```python
 ─ Executing parsed code: ────────────────────────────────────────────────────────────────────────────────────────
  results = web_search(query="best music for a Batman party")                                                      
  print(results)                                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────
```

After a few steps, you'll see the generated playlist that Alfred can use for the party! 🎵

## Using a Custom Tool to Prepare the Menu

Now that we have selected a playlist, we need to organize the menu for the guests. Again, Alfred can take advantage of `smolagents` to do so. Here, we use the `@tool` decorator to define a custom function that acts as a tool. We'll cover tool creation in more detail later, so for now, we can simply run the code.

As you can see in the example below, we will create a tool using `@tool` decorator and include it in the `tools` list.

In [5]:
from smolagents import CodeAgent, tool

@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggests a menu based on the occasion.
    Args:
        occasion (str): The type of occasion for the party. Allowed values are:
                        - "casual": Menu for casual party.
                        - "formal": Menu for formal party.
                        - "superhero": Menu for superhero party.
                        - "custom": Custom menu.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."

agent = CodeAgent(tools=[suggest_menu], model=InferenceClientModel())

agent.run("Prepare a formal menu for the party.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Prepare a formal menu for the party.                                                                            │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  formal_menu = suggest_menu(occasion="formal")                                                                    
  print(formal_menu)                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
3-course dinner with wine and dessert.

Out: None

[Step 1: Duration 0.82 seconds| Input tokens: 2,084 | Output tokens: 47]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  formal_menu = {                                                                                                  
      "Appetizer": "Smoked Salmon with Cream Cheese and Dill",                                                     
      "Main Course": "Herb-Crusted Rack of Lamb with Roasted Vegetables",                                          
      "Dessert": "Chocolate Lava Cake with Vanilla Ice Cream",                                                     
      "Beverages": "Red and White Wine Selection"                                                                  
  }                                                                                                                
                                                                                                                   
  print(formal_menu)                                                                                               
  final_answer(formal_menu)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
{'Appetizer': 'Smoked Salmon with Cream Cheese and Dill', 'Main Course': 'Herb-Crusted Rack of Lamb with Roasted 
Vegetables', 'Dessert': 'Chocolate Lava Cake with Vanilla Ice Cream', 'Beverages': 'Red and White Wine Selection'}

Out - Final answer: {'Appetizer': 'Smoked Salmon with Cream Cheese and Dill', 'Main Course': 'Herb-Crusted Rack of 
Lamb with Roasted Vegetables', 'Dessert': 'Chocolate Lava Cake with Vanilla Ice Cream', 'Beverages': 'Red and White
Wine Selection'}

[Step 3: Duration 1.77 seconds| Input tokens: 6,662 | Output tokens: 240]

{'Appetizer': 'Smoked Salmon with Cream Cheese and Dill',
 'Main Course': 'Herb-Crusted Rack of Lamb with Roasted Vegetables',
 'Dessert': 'Chocolate Lava Cake with Vanilla Ice Cream',
 'Beverages': 'Red and White Wine Selection'}

The agent will run for a few steps until finding the answer.

The menu is ready! 🥗

## Using Python Imports Inside the Agent

We have the playlist and menu ready, but we need to check one more crucial detail: preparation time!

Alfred needs to calculate when everything would be ready if he started preparing now, in case they need assistance from other superheroes.

`smolagents` specializes in agents that write and execute Python code snippets, offering sandboxed execution for security. It supports both open-source and proprietary language models, making it adaptable to various development environments.

**Code execution has strict security measures** - imports outside a predefined safe list are blocked by default. However, you can authorize additional imports by passing them as strings in `additional_authorized_imports`.
For more details on secure code execution, see the official [guide](https://huggingface.co/docs/smolagents/tutorials/secure_code_execution).

When creating the agent, we ill use `additional_authorized_imports` to allow for importing the `datetime` module.

In [6]:
from smolagents import CodeAgent, InferenceClientModel
import numpy as np
import time
import datetime

agent = CodeAgent(tools=[], model=InferenceClientModel(), additional_authorized_imports=['datetime'])

agent.run(
    """
    Alfred needs to prepare for the party. Here are the tasks:
    1. Prepare the drinks - 30 minutes
    2. Decorate the mansion - 60 minutes
    3. Set up the menu - 45 minutes
    3. Prepare the music and playlist - 45 minutes

    If we start right now, at what time will the party be ready?
    """
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Alfred needs to prepare for the party. Here are the tasks:                                                      │
│     1. Prepare the drinks - 30 minutes                                                                          │
│     2. Decorate the mansion - 60 minutes                                                                        │
│     3. Set up the menu - 45 minutes                                                                             │
│     3. Prepare the music and playlist - 45 minutes                                                              │
│                                                                                                                 │
│     If we start right now, at what time will the party be ready?                                                │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import datetime                                                                                                  
                                                                                                                   
  # Define the time required for each task in minutes                                                              
  task_times = {                                                                                                   
      "Prepare the drinks": 30,                                                                                    
      "Decorate the mansion": 60,                                                                                  
      "Set up the menu": 45,                                                                                       
      "Prepare the music and playlist": 45                                                                         
  }                                                                                                                
                                                                                                                   
  # Calculate the total time required for all tasks                                                                
  total_time_minutes = sum(task_times.values())                                                                    
                                                                                                                   
  # Get the current time                                                                                           
  current_time = datetime.datetime.now()                                                                           
                                                                                                                   
  # Calculate the time when the party will be ready                                                                
  party_ready_time = current_time + datetime.timedelta(minutes=total_time_minutes)                                 
                                                                                                                   
  # Print the result                                                                                               
  print("The party will be ready at:", party_ready_time.strftime("%Y-%m-%d %H:%M:%S"))                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
The party will be ready at: 2025-06-21 21:32:44

Out: None

[Step 1: Duration 2.89 seconds| Input tokens: 2,083 | Output tokens: 209]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("2025-06-21 21:32:44")                                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 2025-06-21 21:32:44

[Step 2: Duration 1.15 seconds| Input tokens: 4,630 | Output tokens: 287]

'2025-06-21 21:32:44'

These examples are just the beginning of what you can do with code agents, and we're already starting to see their utility for preparing the party.
You can learn more about how to build code agents in the [smolagents documentation](https://huggingface.co/docs/smolagents).

`smolagents` specializes in agents that write and execute Python code snippets, offering sandboxed execution for security. It supports both open-source and proprietary language models, making it adaptable to various development environments.

## Sharing Our Custom Party Preparator Agent to the Hub

Wouldn't it be **amazing to share our very own Alfred agent with the community**? By doing so, anyone can easily download and use the agent directly from the Hub, bringing the ultimate party planner of Gotham to their fingertips! Let's make it happen! 🎉

The `smolagents` library makes this possible by allowing you to share a complete agent with the community and download others for immediate use. It's as simple as the following:


In [7]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, InferenceClientModel, VisitWebpageTool, FinalAnswerTool, Tool, tool

@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggests a menu based on the occasion.
    Args:
        occasion: The type of occasion for the party.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."

@tool
def catering_service_tool(query: str) -> str:
    """
    This tool returns the highest-rated catering service in Gotham City.

    Args:
        query: A search term for finding catering services.
    """
    # Example list of catering services and their ratings
    services = {
        "Gotham Catering Co.": 4.9,
        "Wayne Manor Catering": 4.8,
        "Gotham City Events": 4.7,
    }

    # Find the highest rated catering service (simulating search query filtering)
    best_service = max(services, key=services.get)

    return best_service

class SuperheroPartyThemeTool(Tool):
    name = "superhero_party_theme_generator"
    description = """
    This tool suggests creative superhero-themed party ideas based on a category.
    It returns a unique party theme idea."""

    inputs = {
        "category": {
            "type": "string",
            "description": "The type of superhero party (e.g., 'classic heroes', 'villain masquerade', 'futuristic gotham').",
        }
    }

    output_type = "string"

    def forward(self, category: str):
        themes = {
            "classic heroes": "Justice League Gala: Guests come dressed as their favorite DC heroes with themed cocktails like 'The Kryptonite Punch'.",
            "villain masquerade": "Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains.",
            "futuristic gotham": "Neo-Gotham Night: A cyberpunk-style party inspired by Batman Beyond, with neon decorations and futuristic gadgets."
        }

        return themes.get(category.lower(), "Themed party idea not found. Try 'classic heroes', 'villain masquerade', or 'futuristic gotham'.")


# Alfred, the butler, preparing the menu for the party
agent = CodeAgent(
    tools=[
        DuckDuckGoSearchTool(),
        VisitWebpageTool(),
        suggest_menu,
        catering_service_tool,
        SuperheroPartyThemeTool()
        ],
    model=InferenceClientModel(),
    max_steps=10,
    verbosity_level=2
)

agent.run("Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme        │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: To create the best playlist for a 'villain masquerade' party at Wayne's mansion, I will first generate a  
theme idea using the `superhero_party_theme_generator` tool. Then, I will use the `web_search` tool to find a      
playlist that matches the generated theme.                                                                         
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
theme_idea = superhero_party_theme_generator(category="villain masquerade")                                        
print(f"Generated theme idea: {theme_idea}")                                                                       
```                                                                                                                

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  theme_idea = superhero_party_theme_generator(category="villain masquerade")                                      
  print(f"Generated theme idea: {theme_idea}")                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Generated theme idea: Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains.

Out: None

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: From the search results, the most relevant playlists for a "villain masquerade" theme at Wayne's mansion  
are:                                                                                                               
1. "Villains Of Gotham City^^A Playlist" on YouTube                                                                
2. "gotham rogues gallery" on Spotify                                                                              
3. "only 𝙫𝙞𝙡𝙡𝙖𝙞𝙣𝙨 allowed [𝘢 𝘨𝘰𝘵𝘩𝘢𝘮 𝘪𝘯𝘴𝘱𝘪𝘳𝘦𝘥 𝘱𝘭𝘢𝘺𝘭𝘪𝘴𝘵]" on YouTube                                                 
4. "Gotham Villains - The Playlist" on YouTube                                                                     
5. "gotham rogues" on Spotify                                                                                      
6. "Arkham Asylum (Batman, Joker, Gotham, Harley Quinn, Scarecrow, villains, rogues)" on Spotify                   
                                                                                                                   
I will choose the "Villains Of Gotham City^^A Playlist" on YouTube as it seems to be a dedicated playlist for      
Gotham villains and should fit the theme well.                                                                     
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
final_answer("https://m.youtube.com/watch?v=DLoJyqxefu4")                                                          
```                                                                                                                

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("https://m.youtube.com/watch?v=DLoJyqxefu4")                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: https://m.youtube.com/watch?v=DLoJyqxefu4

[Step 3: Duration 3.04 seconds| Input tokens: 8,376 | Output tokens: 426]

'https://m.youtube.com/watch?v=DLoJyqxefu4'

In [8]:
agent.push_to_hub(f'{username}/AlfredAgent')

No files have been modified since last commit. Skipping to prevent empty commit.


This agent has step_callbacks: they will be ignored by this method. 1

CommitInfo(commit_url='https://huggingface.co/spaces/gperdrizet/AlfredAgent/commit/bf183d4574d753f20fc6f3786c1e52c3089bf312', commit_message='Upload agent', commit_description='', oid='bf183d4574d753f20fc6f3786c1e52c3089bf312', pr_url=None, repo_url=RepoUrl('https://huggingface.co/spaces/gperdrizet/AlfredAgent', endpoint='https://huggingface.co', repo_type='space', repo_id='gperdrizet/AlfredAgent'), pr_revision=None, pr_num=None)

To download the agent again, use the code below:

In [9]:
agent = CodeAgent(tools=[], model=InferenceClientModel())
alfred_agent = agent.from_hub(f'{username}/AlfredAgent', trust_remote_code=True)

Fetching 14 files: 100%|██████████| 14/14 [00:00<00:00, 25.48it/s]


In [10]:
alfred_agent.run("Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme        │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: To create the best playlist for a 'villain masquerade' party at Wayne's mansion, I will first generate a  
theme idea using the `superhero_party_theme_generator` tool. Then, I will use the `web_search` tool to find a      
playlist that matches the generated theme.                                                                         
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
theme_idea = superhero_party_theme_generator(category="villain masquerade")                                        
print(f"Generated theme idea: {theme_idea}")                                                                       
```                                                                                                                

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  theme_idea = superhero_party_theme_generator(category="villain masquerade")                                      
  print(f"Generated theme idea: {theme_idea}")                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Generated theme idea: Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains.

Out: None

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: The "gotham rogues gallery" playlist by echo seems to fit well with the "villain masquerade" theme,       
featuring songs that evoke a dark and mysterious atmosphere. I will now provide the final answer with the playlist 
URL.                                                                                                               
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
final_answer("https://open.spotify.com/playlist/4ljceMczg5pp7n8RGaFVW4")                                           
```                                                                                                                

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("https://open.spotify.com/playlist/4ljceMczg5pp7n8RGaFVW4")                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: https://open.spotify.com/playlist/4ljceMczg5pp7n8RGaFVW4

[Step 4: Duration 2.63 seconds| Input tokens: 15,061 | Output tokens: 511]

'https://open.spotify.com/playlist/4ljceMczg5pp7n8RGaFVW4'

What's also exciting is that shared agents are directly available as Hugging Face Spaces, allowing you to interact with them in real-time. You can explore other agents [here](https://huggingface.co/spaces/davidberenstein1957/smolagents-and-tools).

For example, the _AlfredAgent_ is available [here](https://huggingface.co/spaces/sergiopaniego/AlfredAgent).

### Inspecting Our Party Preparator Agent with OpenTelemetry and Langfuse 📡

Full trace can be found [here](https://cloud.langfuse.com/project/cm7bq0abj025rad078ak3luwi/traces/995fc019255528e4f48cf6770b0ce27b?timestamp=2025-02-19T10%3A28%3A36.929Z).

As Alfred fine-tunes the Party Preparator Agent, he's growing weary of debugging its runs. Agents, by nature, are unpredictable and difficult to inspect. But since he aims to build the ultimate Party Preparator Agent and deploy it in production, he needs robust traceability for future monitoring and analysis.  

Once again, `smolagents` comes to the rescue! It embraces the [OpenTelemetry](https://opentelemetry.io/) standard for instrumenting agent runs, allowing seamless inspection and logging. With the help of [Langfuse](https://langfuse.com/) and the `SmolagentsInstrumentor`, Alfred can easily track and analyze his agent’s behavior.  

Setting it up is straightforward!  

First, we need to install the necessary dependencies:  

In [11]:
!pip install smolagents[telemetry] opentelemetry-sdk opentelemetry-exporter-otlp openinference-instrumentation-smolagents

Defaulting to user installation because normal site-packages is not writeable


Next, Alfred has already created an account on Langfuse and has his API keys ready. If you haven’t done so yet, you can sign up for Langfuse Cloud [here](https://cloud.langfuse.com/) or explore [alternatives](https://huggingface.co/docs/smolagents/tutorials/inspect_runs).  

Once you have your API keys, they need to be properly configured as follows:

In [12]:
import os
import base64

LANGFUSE_PUBLIC_KEY=os.environ["LANGFUSE_PUBLIC_KEY"]
LANGFUSE_SECRET_KEY=os.environ["LANGFUSE_SECRET_KEY"]
LANGFUSE_AUTH=base64.b64encode(f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}".encode()).decode()

#os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://cloud.langfuse.com/api/public/otel" # EU data region
os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://us.cloud.langfuse.com/api/public/otel" # US data region
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"

Finally, Alfred is ready to initialize the `SmolagentsInstrumentor` and start tracking his agent's performance.  

In [13]:
from opentelemetry.sdk.trace import TracerProvider

from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

trace_provider = TracerProvider()
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter()))

SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

Alfred is now connected 🔌! The runs from `smolagents` are being logged in Langfuse, giving him full visibility into the agent's behavior. With this setup, he's ready to revisit previous runs and refine his Party Preparator Agent even further.  

In [14]:
from smolagents import CodeAgent, InferenceClientModel

agent = CodeAgent(tools=[], model=InferenceClientModel())
alfred_agent = agent.from_hub(f'{username}/AlfredAgent', trust_remote_code=True)
alfred_agent.run("Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme")

Fetching 14 files: 100%|██████████| 14/14 [00:00<00:00, 137196.86it/s]


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme        │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/home/vscode/.local/lib/python3.11/site-packages/smolagents/models.py:315: FutureWarning: Attribute last_input_token_count is deprecated and will be removed in version 1.20. Please use TokenUsage.input_tokens instead.
  warnings.warn(
/home/vscode/.local/lib/python3.11/site-packages/smolagents/models.py:324: FutureWarning: Attribute last_output_token_count is deprecated and will be removed in version 1.20. Please use TokenUsage.output_tokens instead.
  warnings.warn(


Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: To create the best playlist for a 'villain masquerade' party at Wayne's mansion, I will first generate a  
theme idea using the `superhero_party_theme_generator` tool. Then, I will use the `web_search` tool to find a      
playlist that matches the generated theme.                                                                         
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
theme_idea = superhero_party_theme_generator(category="villain masquerade")                                        
print(f"Generated theme idea: {theme_idea}")                                                                       
```                                                                                                                

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: The "gotham rogues gallery" playlist by echo seems to fit well with the "villain masquerade" theme,       
featuring songs that evoke a dark and mysterious atmosphere with references to Batman villains. I will now provide 
the final answer with the playlist URL.                                                                            
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
final_answer("https://open.spotify.com/playlist/4ljceMczg5pp7n8RGaFVW4")                                           
```                                                                                                                

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("https://open.spotify.com/playlist/4ljceMczg5pp7n8RGaFVW4")                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: https://open.spotify.com/playlist/4ljceMczg5pp7n8RGaFVW4

[Step 4: Duration 1.89 seconds| Input tokens: 15,077 | Output tokens: 516]

'https://open.spotify.com/playlist/4ljceMczg5pp7n8RGaFVW4'

Alfred can now access this logs [here](https://cloud.langfuse.com/project/cm7bq0abj025rad078ak3luwi/traces/995fc019255528e4f48cf6770b0ce27b?timestamp=2025-02-19T10%3A28%3A36.929Z) to review and analyze them.  

Meanwhile, the [suggested playlist](https://open.spotify.com/playlist/0gZMMHjuxMrrybQ7wTMTpw) sets the perfect vibe for the party preparations. Cool, right? 🎶
